### **Instalamos dependencias**

In [1]:
%pip install folium
%pip install osmnx networkx scikit-learn tqdm


Note: you may need to restart the kernel to use updated packages.


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### **Comprobamos outliers de ubicación** *(podría estar en el EDA, pero como hacemos viusalización lo incluimos aquí)*

In [2]:
import pandas as pd

df = pd.read_csv("data/processed/rome_clean.csv")

In [3]:
from pathlib import Path
import folium

# Path("maps").mkdir(parents=True, exist_ok=True)

# m = folium.Map(location=[41.9, 12.5], zoom_start=12)
# for _, r in df.iterrows():
#     folium.CircleMarker(location=[r['latO'], r['lonO']],
#                         radius=3, color='blue', fill=True).add_to(m)

# m.save('maps/origins.html')

In [4]:
import webbrowser, os
webbrowser.open('file://' + os.path.abspath('maps/origins.html'))

True

Tenemos que hacer esto para poder mostrar en VSCode el mapa. Observamos que los valores de orígenes están centrados en Roma todos. Vamos a ver los de destino.

In [5]:
# m = folium.Map(location=[41.9, 12.5], zoom_start=12)
# for _, r in df.iterrows():
#     folium.CircleMarker(location=[r['latD'], r['lonD']],
#                         radius=3, color='blue', fill=True).add_to(m)

# m.save('maps/destinations.html')

webbrowser.open('file://' + os.path.abspath('maps/destinations.html'))

True

La distribución de ubicaciones es bastante similar a la de orígenes. El único valor que destaca sobre el resto es un viaje que termino en un parking de Fonte Laurentina (municipio un poco alejado de Roma).

------------------------------------------------------------------------------------------------------------------------------------------------

### ESTIMACION DE RUTAS A TRAVES DE COORDENADAS DE ORIGEN Y DESTINO CON OSM (OpenStreetMaps)

In [6]:
import osmnx as ox
import networkx as nx

#Mapa para la combinación de carreteras y carriles bici en Roma mediante osmnx
# 
# place_name = "Rome, Italy"

# # Grafo para vehículos (carreteras para coches)
# graph_drive = ox.graph_from_place(place_name, network_type='drive')

# # Grafo para carriles bici
# graph_bike = ox.graph_from_place(place_name, network_type='bike')

# combined_graph = nx.compose(graph_drive, graph_bike)

# Guardar el grafo combinado en un archivo GraphML
# ox.save_graphml(combined_graph, filepath="rome_combined.graphml")

# cargar el grafo combinado desde el archivo GraphML
combined_graph= ox.load_graphml("routes/rome_combined.graphml")

### **NOTA[1]: Análisis de las rutas con un tiempo entre 21 y 60 segundos**

Especificado en el EDA. (hay que cambiar el nombre de las variables de df2, para que haya consistencia en el resto del notebook) Ahora mismo hay redundancia en el código que podemos depurar.

In [7]:
import osmnx as ox
import networkx as nx
import pandas as pd

df2 = pd.read_csv("data/processed/rome_clean.csv")


df_filtrado = df2[(df2["tt"] >= 21) & (df2["tt"] <= 60)]

rutas_ajustadas = []
ids_problematicos = []

for index, row in df_filtrado.iterrows():

    latO, lonO = row['latO'], row['lonO']
    latD, lonD = row['latD'], row['lonD']
    id_viaje = row['idS']        
    tiempo_viaje_real = row['tt']
    velocidad_promedio = row['vel']

    # Encontrar nodos más cercanos
    origin_node = ox.distance.nearest_nodes(combined_graph, X=lonO, Y=latO)
    destination_node = ox.distance.nearest_nodes(combined_graph, X=lonD, Y=latD)

    try:

        # Calcular la ruta
        route = nx.shortest_path(
            combined_graph,
            origin_node,
            destination_node,
            weight='length'
        )

        # Calcular distancia total de la ruta
        route_distance = 0
        for u, v in zip(route[:-1], route[1:]):
            edge_data = combined_graph.get_edge_data(u, v)

            if edge_data is None:
                continue

            # Manejo correcto de keys (evita edge_data[0])
            first_key = list(edge_data.keys())[0]

            if 'length' in edge_data[first_key]:
                route_distance += edge_data[first_key]['length']

        # Tiempo estimado
        tiempo_estimado = route_distance / velocidad_promedio

        # Guardamos ruta correcta
        rutas_ajustadas.append({
            'index': index,
            'id_viaje': id_viaje,
            'origin_node': origin_node,
            'destination_node': destination_node,
            'route': route,
            'route_distance': route_distance,
            'tiempo_estimado': tiempo_estimado,
            'tiempo_real': tiempo_viaje_real,
            'diferencia': abs(tiempo_estimado - tiempo_viaje_real)
        })

    except nx.NetworkXNoPath:
        exception_msg = (
            f"No hay ruta entre origen (node {origin_node}) "
            f"y destino (node {destination_node})"
        )

        ids_problematicos.append({
            'index': index,
            'idS': id_viaje,
            'excepcion': exception_msg
        })

    except Exception as e:
        # Captura de cualquier otro error inesperado
        ids_problematicos.append({
            'index': index,
            'idS': id_viaje,
            'excepcion': f"Error inesperado: {str(e)}"
        })

# Convertir listas a DataFrames
rutas_df = pd.DataFrame(rutas_ajustadas)
ids_problematicos_df = pd.DataFrame(ids_problematicos)


Lo tenemos que visualizar en una ventana externa, ya que en VSCode no se ve correctamente.

In [8]:
import folium
from shapely.geometry import LineString

# Crear un mapa centrado en el promedio de los orígenes
m = folium.Map(
    location=[df_filtrado['latO'].mean(), df_filtrado['lonO'].mean()],
    zoom_start=12
)

for idx, row in rutas_df.iterrows():
    route = row['route']
    
    # Obtener coordenadas reales desde nodos del grafo
    coords = [(combined_graph.nodes[n]['y'], combined_graph.nodes[n]['x']) for n in route]
    
    # Añadir polilínea al mapa
    folium.PolyLine(
        coords,
        weight=5,
        opacity=0.7,
        color='blue'
    ).add_to(m)

In [9]:
import webbrowser

# Guardar el mapa
m.save("visualization/rutas_cortas_limpieza.html")

# Abrir automáticamente en el navegador
webbrowser.open("visualization/rutas_cortas_limpieza.html")

True

NOTA: Creo que cuando lo abrimos en el navegador no hace falta especificar en que carpeta está. Échenle un ojo.

Cumple con nuestra teoría. La mayoría de los viajes entre 21 y 60 segundos no son relevantes *(muy poca distancia, además de la lógica de coger un patinete por 1 minuto)*. Por tanto eliminamos estos viajes también. Unos 500 aprox.

In [10]:
df = df[df["tt"] >= 60]
df.to_csv("data/processed/rome_clean.csv", index=False)

Guardamos esos cambio para el resto del Notebook, y ya trabajar con los datos completamente limpios sin outliers.

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### PRUEBA UNITARIA ESTIMACIÓN DE RUTAS (PRIMER VIAJE)

In [11]:
rutas_ajustadas = []

# Iterar sobre cada fila del DataFrame (df)
row = df.iloc[0]
# Extraer las coordenadas de origen y destino, el tiempo y la distancia
latO, lonO = row['latO'], row['lonO']
latD, lonD = row['latD'], row['lonD']
tiempo_viaje_real = row['tt']  # Tiempo real del viaje en segundos
distancia_viaje = row['dis']   # Distancia del viaje en metros
velocidad_promedio = row['vel']  # Velocidad promedio del viaje en m/s

# Encontrar los nodos más cercanos en el grafo
origin_node = ox.distance.nearest_nodes(combined_graph, X=lonO, Y=latO)
destination_node = ox.distance.nearest_nodes(combined_graph, X=lonD, Y=latD)

# Calcular la ruta más corta
route = nx.shortest_path(combined_graph, origin_node, destination_node, weight='length')

# Calcular la distancia de la ruta (en metros)
route_distance = 0
for u, v in zip(route[:-1], route[1:]):
    # Usamos graph.get_edge_data() para obtener los atributos del borde
    edge_data = combined_graph.get_edge_data(u, v)
    if edge_data is not None:
        route_distance += edge_data[0]['length']  # 'length' es el atributo de la distancia
        
        
# Estimar el tiempo para la ruta basada en la velocidad promedi
tiempo_estimado = route_distance / velocidad_promedio  # en segundos

# Comparar el tiempo estimado con el tiempo real del viaje y almacenar el resultado
rutas_ajustadas.append({
    'origin_node': origin_node,
    'destination_node': destination_node,
    'route': route,
    'route_distance': route_distance,
    'tiempo_estimado': tiempo_estimado,
    'tiempo_real': tiempo_viaje_real,
    'diferencia': abs(tiempo_estimado - tiempo_viaje_real)  # Diferencia entre estimado y real
})

# Crear un DataFrame con las rutas ajustadas
rutas_df = pd.DataFrame(rutas_ajustadas)

Visualizamos la ruta en folium

In [12]:
import folium

# Crear un mapa centrado en el punto medio entre origen y destino
map_center = [ (latO + latD) / 2, (lonO + lonD) / 2 ]  # Centro aproximado entre origen y destino
m = folium.Map(location=map_center, zoom_start=13)

# Crear un marcador para el origen y el destino
folium.Marker([latO, lonO], popup="Origen", icon=folium.Icon(color='blue')).add_to(m)
folium.Marker([latD, lonD], popup="Destino", icon=folium.Icon(color='red')).add_to(m)

# Obtener las coordenadas de la ruta (nodos) para la ruta calculada
route_coords = [(combined_graph.nodes[node]['y'], combined_graph.nodes[node]['x']) for node in route]

# Dibujar la ruta en el mapa con una línea conectando los nodos
folium.PolyLine(route_coords, color='green', weight=5, opacity=0.7).add_to(m)

# Añadir marcadores para cada nodo de la ruta
for coord in route_coords:
    folium.CircleMarker(location=coord, radius=5, color='green', fill=True, fill_color='green').add_to(m)

# Mostrar el mapa interactivo
map_filename = 'visualization/ruta_prueba_interactiva.html'
m.save(map_filename)

# Abrir el archivo HTML en tu navegador
import webbrowser
webbrowser.open(map_filename)

True

#### REPETIMOS EL PROCESO (CON LAS 3000 PRIMERAS)

Hacemos try except pues hay allgunas rutas que dan error estimarlas (muy pocas 13/3000). No ejecutar, están guardadas las rutas en el cvs 300_primeras_rutas. Repetir el proceso con las siguientes 3000 (1h aprox) así hasta las 2400. 

In [13]:
# from tqdm import tqdm
# rutas_ajustadas = []
# ids_problematicos=[]
# df_subset = df.head(3000)

# # Iterar sobre cada fila del DataFrame (df)
# for index, row in tqdm(df_subset.iterrows(), total=len(df_subset), desc="Procesando viajes", unit="viaje"):
#     # Extraer las coordenadas de origen y destino, el tiempo y la distancia
#     latO, lonO = row['latO'], row['lonO']
#     latD, lonD = row['latD'], row['lonD']
#     tiempo_viaje_real = row['tt']  # Tiempo real del viaje en segundos
#     distancia_viaje = row['dis']   # Distancia del viaje en metros
#     velocidad_promedio = row['vel']  # Velocidad promedio del viaje en m/s
#     id_viaje = row['idS']  # ID del viaje
#     exception = None

#     # Encontrar los nodos más cercanos en el grafo
#     try:
        
#         origin_node = ox.distance.nearest_nodes(combined_graph, X=lonO, Y=latO)
#         destination_node = ox.distance.nearest_nodes(combined_graph, X=lonD, Y=latD)

#         route = nx.shortest_path(combined_graph, origin_node, destination_node, weight='length')
#         # Calcular la distancia de la ruta (en metros)
#         route_distance = 0
#         for u, v in zip(route[:-1], route[1:]):
#             # Usamos graph.get_edge_data() para obtener los atributos del borde
#             edge_data = combined_graph.get_edge_data(u, v)
#             if edge_data is not None:
#                 route_distance += edge_data[0]['length']  # 'length' es el atributo de la distancia
                
                
#         # Estimar el tiempo para la ruta basada en la velocidad promedi
#         tiempo_estimado = route_distance / velocidad_promedio  # en segundos

#         # Comparar el tiempo estimado con el tiempo real del viaje y almacenar el resultado
#         rutas_ajustadas.append({
#             'origin_node': origin_node,
#             'destination_node': destination_node,
#             'route': route,
#             'route_distance': route_distance,
#             'tiempo_estimado': tiempo_estimado,
#             'tiempo_real': tiempo_viaje_real,
#             'diferencia': abs(tiempo_estimado - tiempo_viaje_real)  # Diferencia entre estimado y real
#         })
        
#     except nx.NetworkXNoPath as e:
#         exception = f"No hay ruta entre origen (ID {origin_node}) y destino (ID {destination_node})"
#         # Si falla la ruta, guardar la excepción y el índice del viaje
#         ids_problematicos.append({
#             'index': index,
#             'idS': id_viaje,
#             'excepcion': exception
#         })

#     except ValueError as e:
#         exception = f"Error en los nodos de origen/destino: {e}"
#         # Si hay un problema con los nodos de origen o destino
#         ids_problematicos.append({
#             'index': index,
#             'idS': id_viaje,
#             'excepcion': exception
#         })

#     except Exception as e:
#         exception = f"Error general: {e}"
#         # Si ocurre cualquier otro error
#         ids_problematicos.append({
#             'index': index,
#             'idS': id_viaje,
#             'excepcion': exception
#         })

# # Crear un DataFrame con las rutas ajustadas
# ids_problematicos_df = pd.DataFrame(ids_problematicos)
# rutas_df = pd.DataFrame(rutas_ajustadas)

In [14]:
# rutas_df.to_csv('3000_primeras_rutas.csv', index=False)
# ids_problematicos_df.to_csv('ids_problematicos.csv', index=False)

Pintamos algunas rutas (50 por ejemplo)

In [15]:
import folium

# 1. Obtener centro aproximado del mapa (centro de Roma)
mapa = folium.Map(location=[41.9028, 12.4964], zoom_start=12)

# 2. Dibujar cada una de las rutas (limitar a 3000)
for count,ruta in enumerate(list(rutas_df['route'])[:50]):

    # Coordenadas de la ruta (nodos → lat/lon)
    coords = [(combined_graph.nodes[n]['y'], combined_graph.nodes[n]['x']) for n in ruta]
    origin_node = ruta[0]  # Primer nodo de la ruta (origen)
    destination_node = ruta[-1]  # Último nodo de la ruta (destino)
    
    origin_coords = (combined_graph.nodes[origin_node]['y'], combined_graph.nodes[origin_node]['x'])
    destination_coords = (combined_graph.nodes[destination_node]['y'], combined_graph.nodes[destination_node]['x'])
    
    # Añadir un marcador para el origen (azul)
    folium.Marker(
        location=origin_coords,
        popup=f"Origen - ID: {count}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(mapa)
    
    # Añadir un marcador para el destino (rojo)
    folium.Marker(
        location=destination_coords,
        popup=f"Destino - ID: {count}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(mapa)

    # Pintar la ruta en verde transparente
    folium.PolyLine(
        coords,
        color="green",
        weight=3,
        opacity=0.4
    ).add_to(mapa)

# 3. Guardar y abrir
mapa.save("visualization/50_rutas_roma.html")
print("Mapa guardado en 50_rutas_roma.html — ábrelo en tu navegador")

Mapa guardado en 50_rutas_roma.html — ábrelo en tu navegador


Prueba Sergio

In [16]:
# ===========================================
#   VISUALIZACIÓN DE 50 RUTAS EN FOLIUM
# ===========================================

import pandas as pd
import ast
import folium

# 1. Cargar las rutas
rutas_df = pd.read_csv("routes/3000_primeras_rutas.csv")

# 2. Convertir columna route de STRING -> LISTA REAL
rutas_df["route"] = rutas_df["route"].apply(ast.literal_eval)

import osmnx as ox
combined_graph = ox.load_graphml("routes/rome_combined.graphml")

# 4. Crear el mapa centrado en Roma
mapa = folium.Map(location=[41.9028, 12.4964], zoom_start=12)

# 5. Dibujar las primeras 50 rutas
for count, ruta in enumerate(rutas_df["route"][:50]):

    # Coordenadas de cada nodo de la ruta
    coords = [(combined_graph.nodes[n]['y'], combined_graph.nodes[n]['x']) for n in ruta]

    # Identificar origen y destino
    origin_node = ruta[0]
    destination_node = ruta[-1]

    origin_coords = (combined_graph.nodes[origin_node]['y'], combined_graph.nodes[origin_node]['x'])
    destination_coords = (combined_graph.nodes[destination_node]['y'], combined_graph.nodes[destination_node]['x'])

    # Marcador origen (azul)
    folium.Marker(
        location=origin_coords,
        popup=f"Origen ruta {count}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(mapa)

    # Marcador destino (rojo)
    folium.Marker(
        location=destination_coords,
        popup=f"Destino ruta {count}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(mapa)

    # Dibujar la ruta
    folium.PolyLine(
        coords,
        color="green",
        weight=3,
        opacity=0.4
    ).add_to(mapa)

# 6. Guardar el mapa
mapa.save("visualization/50_rutas_roma_sergi.html")

print("✔ Mapa guardado como 50_rutas_roma_sergi.html — ábrelo en tu navegador")


✔ Mapa guardado como 50_rutas_roma_sergi.html — ábrelo en tu navegador


In [17]:
# =====================================
#   VISUALIZACIÓN DE 5 RUTAS EN FOLIUM
# ======================================

import pandas as pd
import ast
import folium
import osmnx as ox

# 1. Cargar CSV con rutas
rutas_df = pd.read_csv("routes/3000_primeras_rutas.csv")

# 2. Convertir columna route STRING -> LISTA REAL
rutas_df["route"] = rutas_df["route"].apply(ast.literal_eval)

# 3. Cargar el grafo de Roma (ajusta el archivo si tiene otro nombre)
combined_graph = ox.load_graphml("routes/rome_combined.graphml")

# 4. Crear mapa centrado en Roma
mapa = folium.Map(location=[41.9028, 12.4964], zoom_start=12)

# 5. Colores distintos para cada ruta
colores = ["red", "blue", "green", "purple", "orange"]

# 6. Dibujar las primeras 5 rutas
for idx, ruta in enumerate(rutas_df["route"][:5]):

    # Coordenadas del recorrido (lista de nodos → lat/lon)
    coords = [(combined_graph.nodes[n]['y'], combined_graph.nodes[n]['x']) for n in ruta]

    # Origen y destino
    origin_node = ruta[0]
    destination_node = ruta[-1]

    origin_coords = (combined_graph.nodes[origin_node]['y'], combined_graph.nodes[origin_node]['x'])
    destination_coords = (combined_graph.nodes[destination_node]['y'], combined_graph.nodes[destination_node]['x'])

    # Marcadores de inicio y fin
    folium.Marker(
        location=origin_coords,
        popup=f"Origen ruta {idx}",
        icon=folium.Icon(color='blue', icon='play')
    ).add_to(mapa)

    folium.Marker(
        location=destination_coords,
        popup=f"Destino ruta {idx}",
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(mapa)

    # Dibujar trayectoria completa
    folium.PolyLine(
        coords,
        color=colores[idx],
        weight=5,
        opacity=0.7,
        tooltip=f"Ruta {idx}"
    ).add_to(mapa)

# 7. Guardar mapa
mapa.save("visualization/5_rutas_roma.html")

print("Mapa guardado como 5_rutas_roma.html — abre el archivo para ver las rutas")


Mapa guardado como 5_rutas_roma.html — abre el archivo para ver las rutas


Si vemos el gráfico, podría parecer que hay un error con las rutas, ya que solo aparecen 4 marcas de origen, y 2 de destino, pero lo que ocurre es que se están sobrescribiendo. Eso lo podemos comprobar en la siguiente celda, en la cual el destino de la ruta 1 coincide con el origen de la ruta 2. Esto tiene todo el sentido del mundo, ya que una persona puede acabar una ruta, y dejar el patinente en ese mismo sitio. Esto ocurre sucesivamente con las siguientes rutas.

In [18]:
for idx, ruta in enumerate(rutas_df["route"][:5]):
    origin = ruta[0]
    dest = ruta[-1]
    print(f"Ruta {idx}: origen={origin}, destino={dest}")

Ruta 0: origen=306292511, destino=252418391
Ruta 1: origen=295408368, destino=2390153680
Ruta 2: origen=2390153680, destino=309055380
Ruta 3: origen=309055380, destino=309055380
Ruta 4: origen=309055380, destino=250556547


### Generamos Mapa de Calor de las 1000 primeras rutas sobre Roma *(tarda en visualizarse)*

In [19]:
import pandas as pd
import ast
import folium
from folium.plugins import HeatMap
import osmnx as ox

df = pd.read_csv("routes/3000_primeras_rutas.csv")
df["route"] = df["route"].apply(ast.literal_eval)

G = ox.load_graphml("routes/rome_combined.graphml")

heat_points = []

for ruta in df["route"][:1000]:
    for node in ruta:
        y = G.nodes[node]["y"]
        x = G.nodes[node]["x"]
        heat_points.append([y, x])

m = folium.Map(location=[41.9028, 12.4964], zoom_start=12)
HeatMap(heat_points, radius=6, blur=4).add_to(m)
m